In [1]:
import time
import xlrd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import datetime
from datetime import datetime
from datetime import timedelta
import dateutil.relativedelta
import calendar
import re
import pandas
import pandas as pd

前面都是def funciton，最后把所有function汇集起来




我手头的项目是要爬2010年到现在每个月/每一天的关于同居的微博。目前常见的微博爬虫有三个目标网站，一个是pc端口s.weibo.com，一个是智能手机端口m.weibo.cn，一个是非常老的手机端口weibo.cn。很多微博爬虫都采用智能手机端端口，因为可以爬到很多数据，只要不停往下滑应该就能爬完。但是我目前还没有发现智能手机端口如何搜索指定的日期或时间段，这就不得不用pc端口或者非常老的那个手机端口了。

非常老的手机端口的好处在于不用管验证码，直接就可以登陆，省去了非常多的麻烦。而且不存在‘展开全文’的问题，pc端和智能手机端需要展开原文。而且每一个日期可以搜索到100页，pc端只能搜索到50页。虽然可能pc端每一页比手机端口要多，但是应该还是这个端口能爬到的更多一点。还有一个原因是可以直接勾选原创微博，省去了看上去爬了很多，但实际上爬完了还需要删除所有的转发的过程（虽然复制粘贴的还是要额外删除）。

用selenium是因为微博会反爬虫，selenium可以模拟正常登陆的流程，而且一定要在每一次点击一个页面之后停顿几秒钟（sleep），不然很容易被封号甚至封IP。

In [2]:
def spider_range(YEAR, target_month):
    date = datetime(YEAR, target_month, 1)
    for i in range(31):
        date_reit.append(date.strftime("%Y")+date.strftime("%m")+date.strftime("%d"))
        if date == datetime(YEAR, target_month, 1) + dateutil.relativedelta.relativedelta(months=1)+ dateutil.relativedelta.relativedelta(days=-1):
            break
        else:
            date = date + dateutil.relativedelta.relativedelta(days=1)

In [3]:
def login():   
    #登陆 Login
    driver.get('https://weibo.cn/pub/?vt=' )
   
    elem = driver.find_element_by_xpath("//*[@class='ut']/a[1]").click();
    time.sleep(1)
    
    print("开始自动登陆Loading")
    elem = driver.find_element_by_xpath("//*[@id='loginName']");
    elem.send_keys(username)
    elem = driver.find_element_by_xpath("//*[@id='loginPassword']");
    elem.send_keys(password)
    elem = driver.find_element_by_xpath("//*[@id='loginAction']");
    elem.send_keys(Keys.ENTER)  
    time.sleep(2)
    print('登陆完毕 Login Finish')

In [4]:
#每爬完一天清理一次临时数据
#clear temp data after each day
def clean_temp_data():
    usernames.clear()
    user_links.clear()
    posts.clear()
    likes.clear()
    reblogs.clear()
    comments.clear()
    comment_links.clear()
    post_date.clear()
    post_time.clear()
    post_date_datetime.clear()

In [5]:
#每天保存
#save each day
def save_data():
    for post in posts:
        data['Posts'].append(post)
    for username in usernames:
        data['User Names'].append(username)
    for user_link in user_links:
        data['User Links'].append(user_link)
    for like in likes:
        data['Likes'].append(like)
    for reblog in reblogs:
        data['Reblogs'].append(reblog)
    for comment in comments:
        data['Comments'].append(comment)
    for comment_link in comment_links:
        data['Comment Links'].append(comment_link)
    for post_d in post_date:
        data['Post Date'].append(post_d)
    for post_t in post_time:
        data['Post Time'].append(post_t)
    for post_date_d in post_date_datetime:
        data['Date in Datetime Format'].append(post_date_d)

In [6]:
#翻页 Next Page
def click_next_page():
    NextPage = driver.find_element_by_xpath("//a[contains(text(),'下页')]")
    NextPage.click()
    time.sleep(1)

In [7]:
#开始爬一页微博。注意elements是可以抓取多个，element只能抓一个
#Scraping one page. Notice elements not element. Elements can find multiple element

def scrape_one_page():
    #找到发微博的用户名和连接
    #Get username and href link
    elem = driver.find_elements_by_xpath("//*[@class='nk']")
    for post in elem:
        usernames.append(post.text)
        user_links.append(post.get_attribute('href'))

    #找到微博内容
    #Get post content
    elem = driver.find_elements_by_xpath("//*[@class='ctt']")
    for post in elem:
        posts.append(post.text)

    #点赞
    #likes
    likes_temp = []
    elem = driver.find_elements_by_xpath("//a[contains(text(),'赞[')]")
    for like in elem:
        likes_temp.append(like.text)
    for like in likes_temp:
        like = re.sub(r'\D', '', like) #去除掉‘赞’这个字只保留数字Delete 'likes' text
        likes.append(like)
    
    #转发评论
    #reblogs
    reblog_temp = []
    elem = driver.find_elements_by_xpath("//a[contains(text(),'转发[')]")
    for reblog in elem:
        reblog_temp.append(reblog.text)
    for reblog in reblog_temp:
        reblog = re.sub(r'\D', '', reblog) #保留数字keep the number
        reblogs.append(reblog)  

    #评论和评论的链接comment and links to comment section
    comment_temp = []
    elem = driver.find_elements_by_xpath("//a[contains(text(),'评论[')]")
    for comment in elem:
        comment_temp.append(comment.text)
        comment_links.append(comment.get_attribute('href'))#评论的链接links to comment
    for comment in comment_temp:
        comment = re.sub(r'\D', '', comment) #保留数字keep the number
        comments.append(comment)  

    #发布日期和时间
    elem = driver.find_elements_by_xpath("//*[@class='ct']")
    for time in elem:
        time_txt = time.text.split(' ')
        post_date.append(time_txt[0])
        post_time.append(time_txt[1])
        post_date_datetime.append(datetime(int(time_txt[0][0:4]),
                                           int(time_txt[0][5:7]),
                                           int(time_txt[0][8:10])))

In [83]:
#爬一天内的每一页
#Scrape every page in a day
def scrape_everything_in_a_day(target_page_per_day):    
    
    current_page = []
    max_page = []
    
    #建立两个检测的目标，一个是看有没有内容，一个是看有没有页码。
    #没有内容说明刷新太快被限制访问了，要返回等几秒再重新搜索
    #有内容没有页码说明这一天只有一页，爬下来就结束了
    #有内容有页码就循环一下每一页都爬下来
    #Set up 2 variables. First for checking content, second for page
    #If there is no content, we are refreshing too fast and restrained for access, go back and wait for a couple of seconds
    #If content exists, but page number doesn't. This day only has one page
    #If content exists, and page number also exists, reiterate through days
    
    try_for_content = len(driver.find_elements_by_xpath("//*[@class='nk']"))
    try_for_page = len(driver.find_elements_by_xpath("//*[@class='pa']"))
    #返回的list如果是空的说明没有
    #If list is empty it means no content/page
            
    if try_for_content != 0 and try_for_page == 0:
        #只有一页only one page
        scrape_one_page()
        save_data()
        
    elif try_for_content != 0 and try_for_page != 0:
        elem = driver.find_element_by_xpath("//*[@class='pa']").text.split('/')
        max_page = elem[1]
        max_page = re.sub(r'\D','',max_page)
 
        #如果超过最大页码，目标页码变为最大页码
        #If target page exceeds max page, set it to max
        if int(max_page) <int(target_page_per_day):
            target_page = max_page
        else:
            target_page = target_page_per_day
            
        #开始循环reit
        for page in range(int(target_page) - 1):
            try_for_content = len(driver.find_elements_by_xpath("//*[@class='nk']"))
            try_for_page = len(driver.find_elements_by_xpath("//*[@class='pa']"))
            
            if try_for_content != 0:
                scrape_one_page()
                
            #如果被限制返回等20秒
            #If restrained go back and wait for 20sec
            else:
                driver.back()
                time.sleep(20)
                
                
                elem = driver.find_element_by_xpath("//*[@class='pa']").text.split('/')
                current_page = elem[0]
                current_page = re.sub(r'\D','',current_page)

                elem = driver.find_element_by_xpath("//*[@name='page']")
                jump_one_page = int(current_page) + 1
                elem.clear()
                elem.send_keys(jump_one_page)
                driver.find_element_by_xpath("//*[@value='跳页']").click()
                
                try_for_content = len(driver.find_elements_by_xpath("//*[@class='nk']"))
                try_for_page = len(driver.find_elements_by_xpath("//*[@class='pa']"))
                
                #如果等几秒刷新出来了就爬
                if try_for_content != 0:
                    scrape_one_page()
                #如果还是没有内容说明这一页就是没有内容，我们跳过
                #If still no content, there's no content on this page, skip
                else:
                    driver.back()
                    time.sleep(5)
                    
                    elem = driver.find_element_by_xpath("//*[@class='pa']").text.split('/')
                    current_page = elem[0]
                    max_page = elem[1]
                    current_page = re.sub(r'\D','',current_page)
                    max_page = re.sub(r'\D','',max_page)
                    
                    #判断是否到倒数第二页,倒数第二页跳不了直接结束
                    #if second last page. If true cant' skip, just break
                    if int(current_page) == int(target_page)-1:
                        break
                    #除此之外跳过这一页otherwise skip this page
                    else:
                        elem = driver.find_element_by_xpath("//*[@name='page']")
                        jump_one_page = int(current_page) + 2
                        elem.clear()
                        elem.send_keys(jump_one_page)
                        driver.find_element_by_xpath("//*[@value='跳页']").click()
                        try:
                            driver.find_element_by_xpath("//*[@class='pa']")
                            scrape_one_page()
                        #如果还不行的话我们再返回再等一等
                        except NoSuchElementException:
                            driver.back()
                            time.sleep(30)
                            elem = driver.find_element_by_xpath("//*[@name='page']")
                            elem.clear()
                            elem.send_keys(jump_one_page)
                            driver.find_element_by_xpath("//*[@value='跳页']").click()
                            
                            try:
                                driver.find_element_by_xpath("//*[@class='pa']")
                                scrape_one_page()    
                            except NoSuchElementException:
                                break

            elem = driver.find_element_by_xpath("//*[@class='pa']").text.split('/')
            current_page = elem[0]
            current_page = re.sub(r'\D','',current_page)
            if int(current_page) == int(target_page):
                break
            else:
                click_next_page()
        save_data()
    
    else:
        driver.back()
        driver.back()

In [84]:
#检测搜索完后第一页有没有内容有没有被限流
#test for the first page after reit_search

def test_first_page_in_a_day():
    try_for_content = len(driver.find_elements_by_xpath("//*[@class='nk']"))
    timer = 0
    time.sleep(1)
    while try_for_content == 0:
        timer = timer + 1
        driver.back()
        time.sleep(20) 
        driver.find_element_by_xpath("//*[@type='submit']").click()
        
        try_for_content = len(driver.find_elements_by_xpath("//*[@class='nk']"))
        if try_for_content != 0:
            break
        elif timer == 2:
            break
            #if nothing on that date, break

接下来是搜索前后和按照日期循环

Below are before and after search and reiterate through dates

In [85]:
def open_search_page_first_time():
    #点开搜索页面 Open search tap
    driver.find_element_by_xpath("//a[contains(text(),'搜索')]").click()
    driver.find_element_by_xpath("//a[contains(text(),'高级搜索')]").click()
    time.sleep(1)
    
def keyword_input(key):
    #输入关键词 Input Keywords
    #我这里是"同居" In this case 'cohabitation'
    elem = driver.find_element_by_xpath("//*[@type='text']")
    elem.send_keys(key)
    time.sleep(1)
    
def date_input(day):
    #开始年月日 Start data e.g, 20200101
    elem = driver.find_element_by_xpath("//*[@name='starttime']")
    elem.clear()
    elem.send_keys(day)
    time.sleep(3)

    #结束年月日一样 End data same
    elem = driver.find_element_by_xpath("//*[@name='endtime']")
    elem.clear()
    elem.send_keys(day)
    time.sleep(20)

In [86]:
def search_first_time(key):
    #上面def的两个function
    open_search_page_first_time()
    keyword_input(key)
    
    #勾选原创 Click original posts
    driver.find_element_by_xpath("//*[@name='hasori']").click()
    
    date_input(date_reit[0])
    
    #搜索 Search
    driver.find_element_by_xpath("//*[@type='submit']").click()
    time.sleep(7)
    
def search_reiterate(day):
    #点开‘更多’ Open'more'
    #因为之前输入过关键词和勾选原创，这里可以省略
    #Because we have input keywords and click original, we can skip here
    elem = driver.find_elements_by_xpath("//a[contains(text(),'更多')]")
    
    if elem != 0:
        driver.find_element_by_xpath("//a[contains(text(),'更多')]").click()
        time.sleep(5)
        try:
            date_input(day)
            #搜索 Search
            driver.find_element_by_xpath("//*[@type='submit']").click()
            time.sleep(7)

        except NoSuchElementException:
            driver.back()
            time.sleep(15)

            driver.find_element_by_xpath("//a[contains(text(),'更多')]").click()
            time.sleep(7)

            date_input(day)

            #搜索 Search
            driver.find_element_by_xpath("//*[@type='submit']").click()
            time.sleep(7)
    else:
        driver.back()
        time.sleep(10)
        driver.find_element_by_xpath("//a[contains(text(),'更多')]").click()
        time.sleep(3)
        try:
            date_input(day)
            #搜索 Search
            driver.find_element_by_xpath("//*[@type='submit']").click()
            time.sleep(7)

        except NoSuchElementException:
            driver.back()
            time.sleep(15)

            driver.find_element_by_xpath("//a[contains(text(),'更多')]").click()
            time.sleep(3)

            date_input(day)

            #搜索 Search
            driver.find_element_by_xpath("//*[@type='submit']").click()
            time.sleep(7)

In [87]:
def scrape_by_date(keyword, target_year, 
                   target_month, target_page_per_day):
    
    #创建指定年份的日期列表
    #Create a list of dyas in target year
    spider_range(target_year, target_month)
    
    #登陆
    #login
    login()
    
    #开始搜索指定关键词的第一天的原创微博
    #search for keywords, click original, on the first day in the list
    search_first_time(keyword)
    
    #爬+保存，清除临时数据列表
    scrape_everything_in_a_day(target_page_per_day)
    clean_temp_data()
    print('Finish ' + date_reit[0])
    
    #从第二天开始循环，所以去掉列表里第一天
    #Reit from second day, so delete first day in the list
    reit_from_2nd_day = []
    reit_from_2nd_day = date_reit[1:]
    for day in reit_from_2nd_day:
        search_reiterate(day)
        test_first_page_in_a_day()
        scrape_everything_in_a_day(target_page_per_day)
        print('Finish ' + day)
        clean_temp_data()
    
    print('Finish scraping in ' + str(target_year) + ' ' + str(target_month))

In [114]:
#创建临时数据列表
#Creat temp data
date_reit = []
usernames = []
user_links = []
posts = []
likes = []
reblogs = []
comments = []
comment_links = []
post_date = []   
post_time = []
post_date_datetime = []
data = {'Posts':[],
    'User Names':[],
    'User Links':[],
    'Likes':[],
    'Reblogs':[],
    'Comments':[],
    'Comment Links':[],
    'Post Date':[],
    'Post Time':[],
    'Date in Datetime Format':[]}


#正式跑起来
#running
if __name__ == '__main__':
    username = "17064888409" #用户名username
    password = "rw297421" #密码password        
    keyword = ['同居']
    target_year = 2019
    target_month = 12
    target_page_per_day = 3
    
    driver = webdriver.Chrome()#可以换成firefox.Can use firefox
    for word in keyword:
        scrape_by_date(word, target_year, 
                       target_month, target_page_per_day)
#页数可以设置成max_page

#有时候需要运行几次才能ok，第一次卡在登陆，第二次卡在前两天
#sometimes it takes multiple tries...first time stuck at login, second time at first two days

开始自动登陆Loading
登陆完毕 Login Finish
Finish 20191201
Finish 20191202
Finish 20191203
Finish 20191204
Finish 20191205
Finish 20191206
Finish 20191207
Finish 20191208
Finish 20191209
Finish 20191210
Finish 20191211
Finish 20191212
Finish 20191213
Finish 20191214
Finish 20191215
Finish 20191216
Finish 20191217
Finish 20191218
Finish 20191219
Finish 20191220
Finish 20191221
Finish 20191222
Finish 20191223
Finish 20191224
Finish 20191225
Finish 20191226
Finish 20191227
Finish 20191228
Finish 20191229
Finish 20191230
Finish 20191231
Finish scraping in 2019 12


In [110]:
#检查一下是不是数量相等，相等才能建起来datafram
#check if numbers are equal so that we can turn this into dataframe
print(len(data['User Names']))
print(len(data['User Links']))
print(len(data['Posts']))
print(len(data['Likes']))
print(len(data['Reblogs']))
print(len(data['Comments']))
print(len(data['Comment Links']))
print(len(data['Post Date']))
print(len(data['Post Time']))
print(len(data['Date in Datetime Format']))

484
484
484
484
484
484
484
484
484
484


In [55]:
data_2012_1 = data
df_2012_1 = pd.DataFrame(data_2012_1)
df_2012_1.to_csv(r'data/2019/df_2019_1.csv', index = False, header=True)

In [58]:
data_2012_2 = data
df_2012_2 = pd.DataFrame(data_2012_2)
df_2012_2.to_csv(r'data/2019/df_2019_2.csv', index = False, header=True)

In [63]:
data_2012_3 = data
df_2012_3 = pd.DataFrame(data_2012_3)
df_2012_3.to_csv(r'data/2019/df_2019_3.csv', index = False, header=True)

In [66]:
data_2012_4 = data
df_2012_4 = pd.DataFrame(data_2012_4)
df_2012_4.to_csv(r'data/2019/df_2019_4.csv', index = False, header=True)

In [70]:
data_2012_5 = data
df_2012_5 = pd.DataFrame(data_2012_5)
df_2012_5.to_csv(r'data/2019/df_2019_5.csv', index = False, header=True)

In [73]:
data_2012_6 = data
df_2012_6 = pd.DataFrame(data_2012_6)
df_2012_6.to_csv(r'data/2019/df_2019_6.csv', index = False, header=True)

In [90]:
data_2012_7 = data
df_2012_7 = pd.DataFrame(data_2012_7)
df_2012_7.to_csv(r'data/2019/df_2019_7.csv', index = False, header=True)

In [93]:
data_2012_8 = data
df_2012_8 = pd.DataFrame(data_2012_8)
df_2012_8.to_csv(r'data/2019/df_2019_8.csv', index = False, header=True)

In [95]:
data_2010_9 = data
df_2010_9 = pd.DataFrame(data_2010_9)
df_2010_9.to_csv(r'data/2019/df_2019_9.csv', index = False, header=True)

In [103]:
data_2010_10 = data
df_2010_10 = pd.DataFrame(data_2010_10)
df_2010_10.to_csv(r'data/2019/df_2019_10.csv', index = False, header=True)

In [109]:
data_2010_11 = data
df_2010_11 = pd.DataFrame(data_2010_11)
df_2010_11.to_csv(r'data/2019/df_2019_11.csv', index = False, header=True)

In [115]:
data_2012_12 = data
df_2012_12 = pd.DataFrame(data_2012_12)
df_2012_12.to_csv(r'data/2019/df_2019_12.csv', index = False, header=True)

In [ ]:
df_2010_1 = pd.DataFrame(data_2010_1)
df_2010_2 = pd.DataFrame(data_2010_2)
df_2010_3 = pd.DataFrame(data_2010_3)
df_2010_4 = pd.DataFrame(data_2010_4)
df_2010_5 = pd.DataFrame(data_2010_5)

In [ ]:
df_2010_1.to_csv(r'df_2010_1.csv', index = False, header=True)
df_2010_2.to_csv(r'df_2010_2.csv', index = False, header=True)
df_2010_3.to_csv(r'df_2010_3.csv', index = False, header=True)
df_2010_4.to_csv(r'df_2010_4.csv', index = False, header=True)
df_2010_5.to_csv(r'df_2010_5.csv', index = False, header=True)

In [ ]:
abd = pandas.read_csv('df_2010_1.csv')

In [ ]:
abd

In [ ]:
df_2010_1

In [ ]:
frames = [df_2010_1, df_2010_2, df_2010_3,
         df_2010_4, df_2010_5,]

temp_save_2010_1_5 = pd.concat(frames)

In [ ]:
temp_save_2010_1_5['Posts']

In [ ]:
df_2010_1

In [ ]:
data_2010_5 

In [ ]:
data_2010 = {}

In [ ]:
data_2010 = mergeDict(data, data_2010)
print(len(data_2010['User Names']))

In [ ]:
target_page = 3
for page in range(int(target_page) - 1):
    print(page)

In [ ]:
data_2010['Posts']

In [ ]:
print(len(data_2010['User Names']))

In [ ]:
for key, value in data_2010.items():
    velue = value[:1]

In [ ]:
data_2010.items()

In [ ]:
data_2010['Posts']

In [ ]:
data['Likes'].index('')

In [ ]:
data['Posts'][1409:1411]

In [ ]:
data = pd.DataFrame(data)

In [ ]:
data

In [ ]:
scrape_everything_in_a_day(5)

In [ ]:
#指定年份和季度
#set years and quarter

def spider_range(YEAR, target_month):
    YEAR = int(YEAR)
    date = datetime(YEAR,1,1)
    date_in_target_month = datetime(YEAR, target_month, 1)
    days_in_year = 365 + (1*calendar.isleap(YEAR))
    
    for i in range(days_in_year):
        days_normal.append(date.strftime("%Y")+date.strftime("%m")+date.strftime("%d"))
        date = date + dateutil.relativedelta.relativedelta(days=1) 
    
    for day in days_normal[days_normal.index(str(YEAR) + '0401'):
                     days_normal.index(str(YEAR) + '0701')]:
            reit_dates.append(day)
    
    
    
    if quarter == 1:
        for day in days_normal[:days_normal.index(str(YEAR) + '0401')]:
            reit_dates.append(day)
    
    elif quarter == 2:
        for day in days_normal[days_normal.index(str(YEAR) + '0401'):
                     days_normal.index(str(YEAR) + '0701')]:
            reit_dates.append(day)
    
    elif quarter == 3:
        for day in days_normal[days_normal.index(str(YEAR) + '0701'):
                     days_normal.index(str(YEAR) + '1001')]:
            reit_dates.append(day)

    elif quarter ==4:
        for day in days_normal[days_normal.index(str(YEAR) + '1001'):]:
            reit_dates.append(day)  